**Imports**

In [1]:
!pip install atproto
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.0/357.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.4/682.4 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1


&nbsp;

**Actual Code**

In [2]:
from atproto import Client
from atproto_client import models
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Download stopwords once (only needs to be run the first time)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Connect to Bluesky API
client = Client()
client.login('Spudboy4800@gmail.com', 'P9s1m871QEhlbEJ!!!')

# Fetch posts tagged with #ukraine
def get_ukraine_posts(maxposts):
    allposts = []
    cursor = None
    while True:
        tagged_posts = client.app.bsky.feed.search_posts(
            params=models.AppBskyFeedSearchPosts.Params(
                sort="top",
                q='#ukraine AND lang:en',
                cursor=cursor,
            ))

        for post in tagged_posts.posts:
            allposts += [{
                "author": post.author.display_name,
                "date": post.record.created_at,
                "text": post.record.text,
                "replies": post.reply_count,
                "reposts": post.repost_count,
                "tags": [f.features[0].tag.lower() for f in post.record.facets
                         if f.features[0].py_type == "app.bsky.richtext.facet#tag"] if post.record.facets else []
            }]
        if len(allposts) >= maxposts:
            break
        cursor = tagged_posts.cursor
    return pd.DataFrame(allposts[:maxposts])

# Clean tweet text
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # Remove URLs
    text = re.sub(r"[^\w\s]", '', text)  # Remove punctuation
    text = re.sub(r"\d+", '', text)  # Remove digits
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return " ".join(tokens)

# Run and save posts
allposts = get_ukraine_posts(3000)
allposts.to_json("Devon-Boldt.json")

# Reload posts and clean text
allposts = pd.read_json("Devon-Boldt.json")
allposts['cleaned_text'] = allposts['text'].apply(clean_text)

# Filter by a specific tag
def get_posts_with_tag(posts, tag):
    return posts.loc[posts.tags.apply(lambda x: tag in x)]

# Get all unique tags
def get_all_tags(posts):
    return posts.tags.explode().unique()

# Example usage:
# get_posts_with_tag(allposts, "russia")
# get_all_tags(allposts)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
